# Create an Agent with a single Knowledge Base only

In this notebook you will learn how to create an Amazon Bedrock Agent that connects to a single Knowledge Bases for Amazon Bedrock to retrieve company data and complete tasks. 

The use case for this notebook is the Amazon Bedrock Documentation pages stored as PDFs. It will allow you to ask questions about Amazon Bedrock and get answers based on documents available in the Knowledge Base.

The steps to complete this notebook are:

1. Import the needed libraries
1. Create an S3 bucket and upload the data to it
1. Create the Knowledge Base for Amazon Bedrock and sync data to Knowledge Base
1. Create the Agent for Amazon Bedrock
1. Test the Agent
1. Clean up the resources created

## 1. Import the needed libraries

In [1]:
!pip install --upgrade -q opensearch-py
!pip install --upgrade -q requests-aws4auth
!pip install --upgrade -q boto3
!pip install --upgrade -q botocore
!pip install --upgrade -q awscli

In [2]:
import logging
import boto3
import time
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

[2025-04-21 13:19:53,184] p21885 {credentials.py:1352} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '061051260563')

In [6]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "bedrock-docs-kb-agents"
agent_alias_name = "bedrock-docs-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_arn = f"arn:aws:s3:::{bucket_name}"
bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_bedrock_docs'
kb_name = f'bedrock-docs-kb-{suffix}'
data_source_name = f'bedrock-docs-kb-docs-{suffix}'
kb_files_path = 'kb_documents'
kb_key = 'kb_documents'
kb_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'
kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"
kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"
kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"
kb_collection_name = f'bd-kbc-{suffix}'
# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'

# agent configuration
agent_instruction = """
You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Base
and you can Answer questions from this documentation. Only answer questions based on the documentation and reply with 
"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" 
If the answer to the question is not available in the documentation
"""

You can use a cross Region inference profile in place of a foundation model to route requests to multiple Regions. For the purposes of this lab, we will be using an inference profile to make requests to a foundation model.

To specify this, update the inference profile for the specific model you would like to choose.
- **Amazon Nova Pro**: us.amazon.nova-pro-v1:0
- **Anthropic Claude 3.5 Sonnet v2**: us.anthropic.claude-3-sonnet-20240229-v1:0

In [ ]:
# inference_profile = "us.anthropic.claude-3-sonnet-20240229-v1:0"
inference_profile = "us.amazon.nova-pro-v1:0"
foundation_model = inference_profile[3:] 

print(foundation_model)

amazon.nova-pro-v1:0


## 2. Upload the dataset to Amazon S3
Knowledge Bases for Amazon Bedrock, currently require data to reside in an Amazon S3 bucket. In this section we will create an Amazon S3 bucket and the files.

### 2.1 Create the Amazon S3 bucket

In [8]:
if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name.lower(),
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

### 2.2 Upload dataset to the Amazon S3 bucket

In [9]:
# Upload Knowledge Base files to this s3 bucket
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

## 3. Create a Knowledge Base for Amazon Bedrock

In this section we will go through all the steps to create and test a Knowledge Base. 

These are the steps to complete:
    
1. Create a Knowledge Base Role and its policies
1. Create a Vector Database
1. Create an OpenSearch Index
1. Create a Knowledge Base
1. Create a data source and attach to the recently created Knowledge Base
1. Ingest data to your knowledge Base

### 3.1 Create Knowledge Base Role and Policies

Let's first create IAM policies to allow our Knowledge Base to access Bedrock Titan Embedding Foundation model, Amazon OpenSearch Serverless and the S3 bucket with the Knowledge Base Files.

Once the policies are ready, we will create the Knowledge Base role

In [10]:
# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [11]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [12]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [13]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '62265fa1-b180-4866-8d8d-7c477401e535',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 21 Apr 2025 18:20:06 GMT',
   'x-amzn-requestid': '62265fa1-b180-4866-8d8d-7c477401e535',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [14]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::061051260563:role/AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'

### 3.2 Create Vector Database

Firt of all we have to create a vector store. In this section we will use Amazon OpenSerach Serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.


In [15]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [16]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [17]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:iam::061051260563:user/admin'

In [18]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)


In [19]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

{'createCollectionDetail': {'id': 'cwbzmwmiiaysxzsk4h63',
  'name': 'bd-kbc-us-east-1-061051260563',
  'status': 'CREATING',
  'type': 'VECTORSEARCH',
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'arn': 'arn:aws:aoss:us-east-1:061051260563:collection/cwbzmwmiiaysxzsk4h63',
  'kmsKeyArn': 'auto',
  'standbyReplicas': 'DISABLED',
  'createdDate': 1745259607084,
  'lastModifiedDate': 1745259607084},
 'ResponseMetadata': {'RequestId': '3c213beb-fbd3-4fca-9a9e-11c8abf1a8ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3c213beb-fbd3-4fca-9a9e-11c8abf1a8ae',
   'date': 'Mon, 21 Apr 2025 18:20:07 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '394',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [20]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-east-1:061051260563:collection/cwbzmwmiiaysxzsk4h63'

In [21]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host

Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...

Collection successfully created:
[{'id': 'cwbzmwmiiaysxzsk4h63', 'name': 'bd-kbc-us-east-1-061051260563', 'status': 'ACTIVE', 'type': 'VECTORSEARCH', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'arn': 'arn:aws:aoss:us-east-1:061051260563:collection/cwbzmwmiiaysxzsk4h63', 'kmsKeyArn': 'auto', 'standbyReplicas': 'DISABLED', 'createdDate': 1745259607084, 'lastModifiedDate': 1745259814554, 'collectionEndpoint': 'https://cwbzmwmiiaysxzsk4h63.us-east-1.aoss.amazonaws.com', 'dashboardEndpoint': 'https://cwbzmwmiiaysxzsk4h63.us-east-1.aoss.amazonaws.com/_dashboards'}]


'cwbzmwmiiaysxzsk4h63.us-east-1.aoss.amazonaws.com'

### 3.3 - Create OpenSearch Index

Let's now create a vector index to index our data


In [22]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2025-04-21 13:23:38,226] p21885 {credentials.py:1352} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2025-04-21 13:24:23,991] p21885 {base.py:258} INFO - PUT https://cwbzmwmiiaysxzsk4h63.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:0.758s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


### 3.5 - Create Knowledge Base
Now that we have the Vector database available in OpenSearch Serverless, let's create a Knowledge Base and associate it with the OpenSearch DB

In [23]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [24]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1013',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 21 Apr 2025 18:25:11 GMT',
                                      'x-amz-apigw-id': 'JYsM4GigIAMEVOA=',
                                      'x-amzn-requestid': '8dcb5da1-8dc2-496c-a168-d4686a0642a5',
                                      'x-amzn-trace-id': 'Root=1-68068d85-1e8ac8b55f0360f637466bd9'},
                      'HTTPStatusCode': 202,
                      'RequestId': '8dcb5da1-8dc2-496c-a168-d4686a0642a5',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2025, 4, 21, 18, 25, 9, 386251, tzinfo=tzutc()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:061051260563:knowledge-base/NATQQ

In [25]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': bucket_arn,
    'inclusionPrefixes': [kb_key]  
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '661',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 21 Apr 2025 18:25:56 GMT',
                                      'x-amz-apigw-id': 'JYsUQHWRoAMEtiw=',
                                      'x-amzn-requestid': 'd054a298-deaa-4535-a4c9-83b9ee36193c',
                                      'x-amzn-trace-id': 'Root=1-68068db4-008bb713014c7cfe5fa58c1f'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd054a298-deaa-4535-a4c9-83b9ee36193c',
                      'RetryAttempts': 0},
 'dataSource': {'createdAt': datetime.datetime(2025, 4, 21, 18, 25, 56, 523972, tzinfo=tzutc()),
                'dataDeletionPolicy': 'DELETE',
                'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-docs-kb-agents-us-east-1-0610512605

### 3.6 - Start ingestion job

Once the Knowledge Base and Data Source are created, we can start the ingestion job. During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Amazon OpenSource Serverless.


In [26]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id, 
    dataSourceId=data_source_id
)


## 4. Create Agent

We will now create the Agent and associate the Knowledge Base to it. To do so we need to: 
1. Create Agent IAM role and policies
1. Create Agent
1. Associate Agent to Knowledge Base
1. Prepare Agent

### 4.1 - Create Agent IAM role and policies
First we need to create the agent policies that allow bedrock model invocation and Knowledge Base retrieval

In [ ]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": "*"
        }

    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [28]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)


In [29]:

# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '69d37eb2-46cf-41dc-a260-e67161dfdeba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 21 Apr 2025 18:26:08 GMT',
   'x-amzn-requestid': '69d37eb2-46cf-41dc-a260-e67161dfdeba',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### 4.2 - Create Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [ ]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=inference_profile,
    instruction=agent_instruction,
)

time.sleep(30)

ConflictException: An error occurred (ConflictException) when calling the CreateAgent operation: Could not perform Create operation, since the bedrock-docs-kb-agents (id: Q9XRUYJWSW) with the same name bedrock-docs-kb-agents already exists

Let's now store the agent id in a local variable to use it on the next steps

In [35]:
agent_id = response['agent']['agentId']
agent_id

'Q9XRUYJWSW'

### 4.3 - Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [36]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

### 4.4 - Prepare Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [37]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': 'e540bb0d-4b0b-45a6-a847-e3ccd642996d',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 21 Apr 2025 18:26:42 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e540bb0d-4b0b-45a6-a847-e3ccd642996d',
   'x-amz-apigw-id': 'JYsbcH79IAMEfVg=',
   'x-amzn-trace-id': 'Root=1-68068de2-69ab2f557839f0446c102b70'},
  'RetryAttempts': 0},
 'agentId': 'Q9XRUYJWSW',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 4, 21, 18, 26, 42, 788726, tzinfo=tzutc())}

## 5 - Testing Agent

Now that we have our agent, let's invoke it to test if it is providing correct information about Amazon Bedrock. To do so, let's first create an Agent Alias

In [38]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [39]:
agent_alias

{'ResponseMetadata': {'RequestId': '1484481d-84c8-4f9e-bc1b-d0be5d75d11d',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 21 Apr 2025 18:27:14 GMT',
   'content-type': 'application/json',
   'content-length': '344',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1484481d-84c8-4f9e-bc1b-d0be5d75d11d',
   'x-amz-apigw-id': 'JYsgbHzFoAMEagw=',
   'x-amzn-trace-id': 'Root=1-68068e02-252383cf696f235a14abeb2b'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:061051260563:agent-alias/Q9XRUYJWSW/E49LPXJKBE',
  'agentAliasId': 'E49LPXJKBE',
  'agentAliasName': 'bedrock-docs-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'Q9XRUYJWSW',
  'createdAt': datetime.datetime(2025, 4, 21, 18, 27, 14, 476458, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2025, 4, 21, 18, 27, 14, 476458, tzinfo=tzutc())}}

Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and get the information from the Knowledge base

In [40]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How can I evaluate models on Bedrock?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))


[2025-04-21 13:27:44,770] p21885 {2021949151.py:19} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 21 Apr 2025 18:27:44 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '515abe06-1ede-11f0-953e-a217aa703a64',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '39818cdf-6082-42ae-bd07-e50706463b38'},
                      'HTTPStatusCode': 200,
                      'RequestId': '39818cdf-6082-42ae-bd07-e50706463b38',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x113d47a10>,
 'contentType': 'application/json',
 'sessionId': '515abe06-1ede-11f0-953e-a217aa703a64'}


In [41]:
%%time
from datetime import datetime


class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)


event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2, cls=DateTimeEncoder))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-04-21 13:27:44,979] p21885 {<timed exec>:21} INFO - {
  "agentAliasId": "E49LPXJKBE",
  "agentId": "Q9XRUYJWSW",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:061051260563:agent-alias/Q9XRUYJWSW/E49LPXJKBE"
    }
  ],
  "eventTime": "2025-04-21T18:27:45.025461+00:00",
  "sessionId": "515abe06-1ede-11f0-953e-a217aa703a64",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledgeBaseId": "NATQQK3CYH",
          "text": "How can I evaluate models on Bedrock?"
        },
        "traceId": "39818cdf-6082-42ae-bd07-e50706463b38-0"
      }
    }
  }
}
[2025-04-21 13:27:45,468] p21885 {<timed exec>:21} INFO - {
  "agentAliasId": "E49LPXJKBE",
  "agentId": "Q9XRUYJWSW",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:061051260563:agent-alias/Q9XRUYJWSW/E49LPXJKBE"
    }
  ],


CPU times: user 12.3 ms, sys: 5.37 ms, total: 17.7 ms
Wall time: 9.45 s


In [42]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

To evaluate models on Amazon Bedrock, you can create an automatic model evaluation job. Here are the steps to do so:

1. Open the Amazon Bedrock console: https://console.aws.amazon.com/bedrock/
2. In the navigation pane, choose "Model evaluation."
3. In the "Build an evaluation" card, under "Automatic," choose "Create automatic evaluation."
4. On the "Create automatic evaluation" page, provide the following information:
   a. Evaluation name — Give the model evaluation job a name that describes the job. This name must be unique in your AWS account in an AWS Region.
   b. Description (Optional) — Provide an optional description.
   c. Models — Choose the model you want to use in the model evaluation job.
   d. (Optional) To change the inference configuration, choose "Update."
   e. Task type — Choose the type of task you want the model to attempt to perform during the model evaluation job.
   f. Metrics and datasets — Choose from the list of available metrics and built-in prompt dataset

In [43]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    if session_id is None:
        session_id:str = str(uuid.uuid1())

    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    agent_answer = ''
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    return agent_answer

In [44]:
simple_agent_invoke("what is bedrock provisioned throughput?", agent_id, agent_alias_id, session_id)

[2025-04-21 13:27:54,316] p21885 {321621934.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 21 Apr 2025 18:27:54 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '515abe06-1ede-11f0-953e-a217aa703a64',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '7d8deed5-2718-4914-8604-4662159b904c'},
                      'HTTPStatusCode': 200,
                      'RequestId': '7d8deed5-2718-4914-8604-4662159b904c',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x1148af9d0>,
 'contentType': 'application/json',
 'sessionId': '515abe06-1ede-11f0-953e-a217aa703a64'}


[2025-04-21 13:28:01,862] p21885 {321621934.py:21} INFO - Final answer ->
Bedrock Provisioned Throughput refers to the ability to provision a higher level of throughput for a model at a fixed cost. Throughput is the number and rate of inputs and outputs that a model processes and returns. You can purchase Provisioned Throughput to ensure a consistent level of performance for your model.

Key points about Bedrock Provisioned Throughput:
- You're billed hourly for the Provisioned Throughput you purchase.
- The price per hour depends on the model, the number of Model Units (MUs) specified, and the commitment duration.
- MUs deliver a specific throughput level, specifying the number of input and output tokens an MU can process and generate within a minute.
- Commitment durations include no commitment, 1 month, and 6 months.
- Provisioned Throughput is supported in specific regions and for certain models.
- After purchasing Provisioned Throughput, you can use it to run model inference, eith

"Bedrock Provisioned Throughput refers to the ability to provision a higher level of throughput for a model at a fixed cost. Throughput is the number and rate of inputs and outputs that a model processes and returns. You can purchase Provisioned Throughput to ensure a consistent level of performance for your model.\n\nKey points about Bedrock Provisioned Throughput:\n- You're billed hourly for the Provisioned Throughput you purchase.\n- The price per hour depends on the model, the number of Model Units (MUs) specified, and the commitment duration.\n- MUs deliver a specific throughput level, specifying the number of input and output tokens an MU can process and generate within a minute.\n- Commitment durations include no commitment, 1 month, and 6 months.\n- Provisioned Throughput is supported in specific regions and for certain models.\n- After purchasing Provisioned Throughput, you can use it to run model inference, either through the console or API.\n\nFor detailed information, refer

In [45]:
simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

[2025-04-21 13:28:01,960] p21885 {321621934.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 21 Apr 2025 18:28:02 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '515abe06-1ede-11f0-953e-a217aa703a64',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '38c06f21-a1be-49e0-adc8-e239e0fd6b8b'},
                      'HTTPStatusCode': 200,
                      'RequestId': '38c06f21-a1be-49e0-adc8-e239e0fd6b8b',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x1148af9d0>,
 'contentType': 'application/json',
 'sessionId': '515abe06-1ede-11f0-953e-a217aa703a64'}


Exception: ('unexpected event.', EventStreamError('An error occurred (throttlingException) when calling the InvokeAgent operation: Your request rate is too high. Reduce the frequency of requests. Check your Bedrock model invocation quotas to find the acceptable frequency.'))

In [ ]:
simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

## 6 - Clean up (Optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. delete agent alias
1. delete agent
1. delete the knowledge base
1. delete the OpenSearch Serverless vector store
1. empty created s3 bucket
1. delete s3 bucket


In [46]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [47]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [48]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'VF9B8VGTHSA8TQ6R',
  'HostId': '55HK1gnUh9yrnVq6u14f4H2zbifF9G3uAPo5gzggVZCe5HYM4/tx2oWcceYprK/VDWjXQU3+owH+KKjQEA/a5g+UPSZEqf9Z',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '55HK1gnUh9yrnVq6u14f4H2zbifF9G3uAPo5gzggVZCe5HYM4/tx2oWcceYprK/VDWjXQU3+owH+KKjQEA/a5g+UPSZEqf9Z',
   'x-amz-request-id': 'VF9B8VGTHSA8TQ6R',
   'date': 'Mon, 21 Apr 2025 18:28:15 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [49]:
# Delete IAM Roles and policies and Knowledge Base files
for policy in [
    agent_bedrock_policy, 
    agent_kb_schema_policy,
    kb_bedrock_policy,
    kb_aoss_policy,
    kb_s3_policy
]:
    response = iam_client.list_entities_for_policy(
        PolicyArn=policy['Policy']['Arn'],
        EntityFilter='Role'
    )

    for role in response['PolicyRoles']:
        iam_client.detach_role_policy(
            RoleName=role['RoleName'], 
            PolicyArn=policy['Policy']['Arn']
        )

    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

    

for role_name in [
    agent_role_name, 
    kb_role_name
]:
    try: 
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as e:
        print(e)
        print("couldn't delete role", role_name)
        
    
try:

    open_search_serverless_client.delete_collection(
        id=opensearch_collection_response["createCollectionDetail"]["id"]
    )

    open_search_serverless_client.delete_access_policy(
          name=kb_collection_name,
          type='data'
    )    

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='network'
    )   

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='encryption'
    )    
    bedrock_agent_client.delete_knowledge_base(
        knowledgeBaseId=knowledge_base_id
    )
except Exception as e:
    print(e)

## Conclusion

We have now experimented with using boto3 SDK to create, invoke and delete an agent having a single KB connected to it.
## Take aways

Adapt this notebook to create new agents for your application

## Thank You